In [1]:
import autograd
import autograd.numpy as np
import autograd.scipy as sp
from autograd.test_util import check_grads

import paragami
import scipy as osp

import copy
import example_utils

In [2]:
np.random.seed(42)

num_obs = 1000

# True values of parameters
true_sigma = \
    np.eye(3) * np.diag(np.array([1, 2, 3])) + \
    np.random.random((3, 3)) * 0.1
true_sigma = 0.5 * (true_sigma + true_sigma.T)

true_mu = np.array([0, 1, 2])

# Data
x = np.random.multivariate_normal(
    mean=true_mu, cov=true_sigma, size=(num_obs, ))

# Original weights.
original_weights = np.ones(num_obs)

# Original regularizer.
original_lam = 1.0

In [3]:
def model_logpdf(norm_par, weights, x):
    data_lpdf = example_utils.get_normal_log_prob(
        x, mu=norm_par['mu'], sigma=norm_par['sigma'])
    return np.sum(weights * data_lpdf)

def objective_fun(norm_par, weights, lam, x):
    return \
        -model_logpdf(norm_par, weights, x) + \
        example_utils.regularizer(norm_par, lam)

norm_par = dict()
norm_par['mu'] = true_mu
norm_par['sigma'] = true_sigma

objective_fun(norm_par, original_weights, original_lam, x)

2397.751922600241

In [4]:
norm_param_pattern = paragami.PatternDict()
norm_param_pattern['sigma'] = paragami.PSDSymmetricMatrixPattern(size=3)
norm_param_pattern['mu'] = paragami.NumericArrayPattern(shape=(3, ))

objective_flat = paragami.FlattenFunctionInput(
    lambda par_dict: objective_fun(par_dict, original_weights, original_lam, x),
    patterns=norm_param_pattern,
    free=True)

# It's always a good idea to double check for bugs.

# This checks that the two functions return the same value.
norm_par_flat = norm_param_pattern.flatten(norm_par, free=True)
assert(
    objective_flat(norm_par_flat) == \
    objective_fun(norm_par, original_weights, original_lam, x))

# This verifies that the objective is being differentiated correctly.
check_grads(objective_flat)(norm_par_flat)

In [5]:
objective_wrapper = paragami.OptimizationObjective(objective_flat)
objective_wrapper.set_print_every(5)

init_param = np.ones(norm_param_pattern.flat_length(free=True))
mle_opt = osp.optimize.minimize(
    method='trust-ncg',
    x0=init_param,
    fun=objective_wrapper.f,
    jac=objective_wrapper.grad,
    hessp=objective_wrapper.hessian_vector_product,
    options={'gtol': 1e-12, 'disp': False})

Iter 0: f = 3594.52979620
Iter 5: f = 2462.38498397
Iter 10: f = 2390.42549495
Iter 15: f = 2390.42528542


In [6]:
print(mle_opt.x)
norm_par_opt = norm_param_pattern.fold(mle_opt.x, free=True)
print(norm_par_opt['sigma'])
print(true_sigma)
print(norm_par_opt['mu'])
print(true_mu)

par_flat0 = copy.deepcopy(mle_opt.x)
hess0 = objective_wrapper.hessian(par_flat0)

[ 0.03234828  0.07658348  0.31752785  0.04095079 -0.02154567  0.5360903
 -0.04491706  1.02715611  1.84439126]
[[ 1.06683526  0.07910132  0.04229714]
 [ 0.07910132  1.89299228 -0.02646173]
 [ 0.04229714 -0.02646173  2.92388492]]
[[1.03745401 0.07746864 0.03950388]
 [0.07746864 2.01560186 0.05110853]
 [0.03950388 0.05110853 3.0601115 ]]
[-0.04491706  1.02715611  1.84439126]
[0 1 2]


In [7]:
# To evaluate the regularization parameter
lam_pattern = paragami.NumericScalarPattern(lb=0)
objective_reg_flat = \
    objective_flat = paragami.FlattenFunctionInput(
        lambda par_dict, lam: objective_fun(par_dict, original_weights, lam, x),
    patterns=[norm_param_pattern, lam_pattern],
    free=True)
lam_flat0 = lam_pattern.flatten(original_lam, free=True)

# Sanity check
assert(objective_reg_flat(par_flat0, lam_flat0) ==
       objective_fun(norm_par_opt, original_weights, original_lam, x))

weight_sens = paragami.ParametricSensitivityTaylorExpansion(
    objective_reg_flat,
    input_val0=par_flat0,
    hyper_val0=lam_flat0,
    hess0=hess0,
    order=4)


2390.4252854239494

Using default hessian.
(9, 9)
[[ 2.00371604e+03 -4.08083137e+01 -2.36616455e-07 -1.44269783e+01
  -2.64059828e-08  6.25590901e-09  1.19676507e-01 -8.79817127e-03
  -3.11042063e-03]
 [-4.08083137e+01  5.29990163e+02  3.08130614e-06  5.36806012e+00
  -4.26539266e-08 -2.32772913e-09  1.98040540e+00  1.14264561e-01
   1.15734041e-03]
 [-2.36616454e-07  3.08130618e-06  2.00015888e+03  1.56117982e-08
   7.37425185e+00  7.67234717e-10 -3.01173001e-01  4.05108941e+00
   2.02245489e-02]
 [-1.44269783e+01  5.36806012e+00  1.56117981e-08  3.42261365e+02
   7.24590846e-07 -1.48413321e-07  3.56835501e+00  1.15734041e-03
   7.37906990e-02]
 [-2.64059825e-08 -4.26539221e-08  7.37425185e+00  7.24590849e-07
   3.42261365e+02  3.56096863e-08 -2.37406747e-01  2.69995863e+00
   9.38682577e-01]
 [ 6.25590886e-09 -2.32772882e-09  7.67234615e-10 -1.48413326e-07
   3.56096874e-08  2.00000000e+03 -3.01080000e-01  1.15710450e-01
   7.37756575e+00]
 [ 1.19676507e-01  1.98040540e+00 -3.01173001e-01  3.56835501e+0